# Imports



In [ ]:
from IPython.core.display import display
!pip install pandas
!pip install numpy
!pip install jupyter-dash
!pip install matplotlib
!pip install plotly

import pandas as pd
import os.path


Load track features

In [ ]:
featureData = pd.read_csv('data/track_features.csv')

Function to import data by region group

In [ ]:
def load_charts_for_codes(codes):
    temp_df = pd.DataFrame()
    for code in codes["code"]:
        if os.path.isfile("data/chart_{}.csv".format(code.lower())):
            temp_df = temp_df.append(pd.read_csv("data/chart_{}.csv".format(code.lower())))
    return temp_df

Load regional chart data

In [ ]:
#Europe
codes = pd.read_csv("data/EuropeCode.csv")
chartDataEurope = load_charts_for_codes(codes)

#Asia
codes = pd.read_csv('data/AsiaCode.csv')
chartDataAsia = load_charts_for_codes(codes)

#NorthAmerica
codes = pd.read_csv('data/NorthAmericaCode.csv')
chartDataNorthAmerica = load_charts_for_codes(codes)

#SouthAmerica
codes = pd.read_csv('data/SouthAmericaCode.csv')
chartDataSouthAmerica = load_charts_for_codes(codes)

#Africa
codes = pd.read_csv('data/AfricaCode.csv')
chartDataAfrica = load_charts_for_codes(codes)

#Oceania
codes = pd.read_csv('data/OcianiaCode.csv')
chartDataOceania = load_charts_for_codes(codes)

Append to create a global chart dataframe

In [ ]:
globalChartData = chartDataEurope
globalChartData = globalChartData.append(chartDataAsia)
globalChartData = globalChartData.append(chartDataNorthAmerica)
globalChartData = globalChartData.append(chartDataSouthAmerica)
globalChartData = globalChartData.append(chartDataAfrica)
globalChartData = globalChartData.append(chartDataOceania)

Filter for one datapoint per month, because a global, daily map will cause performance issues in cloropleth map

Monthly or bi-weekly groupby/mean might be better

In [ ]:
globalChartDataMonthly = globalChartData[globalChartData.date.str.endswith("01")]


# Merging

## prepare data of countries

Function to join chart tracks and track features

In [ ]:
def join_chart_features_regions(chart):
    info = chart.merge(featureData, left_on='track_id', right_on='track_id')
    info = info.groupby(['country', 'date'], as_index=False).mean()
    return info


def join_chart_features_continents(chart):
    info = chart.merge(featureData, left_on='track_id', right_on='track_id')
    info = info.groupby('date', as_index=False).mean()
    return info

Create monthly chart dataframe with features of all spotify regions

In [ ]:
newglobalChartDataMonthly = join_chart_features_regions(globalChartDataMonthly)

Replace country codes with full country name, needed by Dash cloropleth map

In [ ]:
import csv

def createCountryCodeDict(file):
    with open(file, mode='r') as infile:
        next(infile) #skip first line (name,code,)
        reader = csv.reader(infile)
        dict = {rows[1].lower():rows[0] for rows in reader}
        return dict

countryCodeNameDict = createCountryCodeDict("data/AfricaCode.csv")
countryCodeNameDict.update(createCountryCodeDict("data/AsiaCode.csv"))
countryCodeNameDict.update(createCountryCodeDict("data/EuropeCode.csv"))
countryCodeNameDict.update(createCountryCodeDict("data/NorthAmericaCode.csv"))
countryCodeNameDict.update(createCountryCodeDict("data/SouthAmericaCode.csv"))
countryCodeNameDict.update(createCountryCodeDict("data/OcianiaCode.csv"))


for key, value in countryCodeNameDict.items():
    newglobalChartDataMonthly['country'] = newglobalChartDataMonthly['country'].replace([key],value)

#newglobalChartDataMonthly.to_csv('data/newglobalChartDataMonthly.csv')

# Create Plotly Dash web app

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

#newglobalChartDataMonthly = pd.read_csv('data/newglobalChartDataMonthly.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
server = app.server

spotify_params = ["danceability",	"energy",	"key", "loudness", "mode", "speechiness", "acousticness",	"instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
colorscales = px.colors.named_colorscales()

app.layout = html.Div([
    html.P("Audio feature"),
    dcc.Dropdown(id='var_choice'  , value='danceability'  , options=[{'label': i, 'value': i} for i in  spotify_params]),
    html.P("Color Scale"),
    dcc.Dropdown(
        id='colorscale',
        options=[{"value": x, "label": x}
                 for x in colorscales],
        value='viridis'
    ),
    dcc.Graph(id='my-graph'),
])

@app.callback(     Output('my-graph'        , 'figure'    ) ,
                  [Input("colorscale", "value"), Input('var_choice'       , 'value'     )] )
def update_figure(scale, var_choice):


    fig = px.choropleth(newglobalChartDataMonthly, locations="country",
                    locationmode="country names",
                    color=newglobalChartDataMonthly[var_choice],
                    animation_frame=newglobalChartDataMonthly["date"],
color_continuous_scale=scale,
                 width=1400, height=800)

    return fig

if __name__ == '__main__':
    app.run_server(port=8056, debug=True  )